In [91]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone 
from langchain.vectorstores import Pinecone
from langchain import OpenAI
from langchain.chains import RetrievalQA
import json
import glob
import os
import random

In [65]:
PINECONE_API_KEY = "PINECONE_API_KEY"
PINECONE_ENV = "PINECONE_ENV next to API_KEY"
OPENAI_API_KEY = "OPENAI_API_KEY"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [82]:
json_files = glob.glob('../data/*.json')

data = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

data = [{'price': e['price'],
         'title': e['title'],
         'size_and_rooms': e['size_and_rooms'],
         'location': e['location'],
         'description': e['description'],
    } for e in data]

for i in range(len(data)):
    data[i]['price'] = data[i]['price'].replace('\xa0', ' ')
    for e in ['title', 'size_and_rooms', 'location', 'description']:
        try:
            data[i][e] = data[i][e].encode('latin1').decode('utf8')
        except: pass

data = [' '.join([f'"{key}": {e[key]};' for key in e.keys()]) for e in data]

In [85]:
data = random.sample(data, 200)

In [87]:
embeddings = OpenAIEmbeddings()
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

data_db = Pinecone.from_texts(
    data, 
    embeddings, 
    index_name='langchain-bot'
)

In [89]:
query = "une maison de 60m2 avec un prix qui ne dépasse pas 1200000DH"
search_docs = data_db.similarity_search(query)
search_docs

[Document(page_content='"price": 2 625 000 DH; "title": Joli Appt à vendre sur BD MOULAY IDRISS I; "size_and_rooms": None; "location": Mers Sultan à Casablanca; "description": nous vous apportons une grande opportunité pour les familles qui cherchent la stabilité au centre de casablanca avec un prix négociable. dans une résidence calme et propre, avec ascenseur, on trouve ce ... En savoir plus;', metadata={}),
 Document(page_content='"price": 1 200 000 DH; "title": Opportunité à ne pas rater; "size_and_rooms": None; "location": Ain Chock à Casablanca; "description": bonjour appartement à vendre à côté du bd al qods113m2 bien ensoleillé 4ème etage. 3 chambres dont une suite parentale , doubke salon, séjour, 2sdb wc, cuisine équipée . prix : 1 200 000 dh télé :;', metadata={}),
 Document(page_content='"price": 1 120 000 DH; "title": Appartement 98m² 5 min de Casanearshore; "size_and_rooms": None; "location": Sidi Maarouf à Casablanca; "description": je mets en vente un appartement de 98m

In [99]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=data_db.as_retriever(),
)

query = "une maison de 60m2 avec un prix qui ne dépasse pas 1200000DH"
result = qa.run(query)

result

'Désolé, je n\'ai pas d\'informations sur une maison de 60m2 avec un prix qui ne dépasse pas 1200000DH. Cependant, voici une annonce qui correspond à votre budget : "Joli studio avec terrasse à vendre à Maarif" avec une taille de 61 m² et un prix de 1 000 000 DH.'